# Working through masked arrays

= as a way to remove the nans in arrays

In [3]:
# While in argo environment: Import necessary packages for this notebook 
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline 
import glob
#!python -m pip install "dask[complete]" 

### Read in data
This reads in the low res data I downloaded in early 2021, the Dec 2021 snapshot data and some high res data I have hanging out in my raw data folder(...)

In [37]:
float_id = '5904183' # '9094'/5904471 '9099' '7652' '9125'
f_id21='5904183'
rootdirLR = '../data/raw/LowRes/'
rootdir21 = '../data/raw/SOCCOM_HiResQC_LIAR_21Dec2021_netcdf/'
rootdir = '../data/raw/'
fd = xr.open_mfdataset(rootdir + float_id + '*.nc') #SOOCNQC
fd21=xr.open_mfdataset(rootdir21 + f_id21 + '*.nc') #SOOCNQC
#fdLR = xr.open_mfdataset(rootdirLR + float_id + '*.nc') #SOOCNQC


#JULD = pd.to_datetime(fd.JULD.values)
#JULD21 = pd.to_datetime(fd21.JULD.values)

In [31]:
print(fd) #fd21 # fdLR

<xarray.Dataset>
Dimensions:              (N_PROF: 181, N_LEVELS: 557, NPARAMETER: 46)
Dimensions without coordinates: N_PROF, N_LEVELS, NPARAMETER
Data variables: (12/66)
    Cruise               |S11 ...
    Station              (N_PROF) int32 dask.array<chunksize=(181,), meta=np.ndarray>
    Lon                  (N_PROF) float64 dask.array<chunksize=(181,), meta=np.ndarray>
    Lat                  (N_PROF) float64 dask.array<chunksize=(181,), meta=np.ndarray>
    Lat_QF               (N_PROF) |S1 dask.array<chunksize=(181,), meta=np.ndarray>
    Lat_QFA              (N_PROF) float64 dask.array<chunksize=(181,), meta=np.ndarray>
    ...                   ...
    Type                 |S1 ...
    mon_day_yr           (N_PROF) |S10 dask.array<chunksize=(181,), meta=np.ndarray>
    hh_mm                (N_PROF) |S5 dask.array<chunksize=(181,), meta=np.ndarray>
    Parameters           (NPARAMETER) |S19 dask.array<chunksize=(46,), meta=np.ndarray>
    JULD                 (N_PROF) dateti

#### Non-masked array method for removing nans
Find either the nan values or the non-nan values.
Then, like with matlab, make a variable where the non-nan values are selected. 

In [6]:
#print(np.array(fd.Chl_a[0]))
#print(np.array(fd.Chl_a_QFA[0])) # not sure about these numbers...
chl_test=np.isfinite(np.array(fd.Chl_a[1])) #non-nan values
#print(chl_test)

chl_test2=np.array(fd.Chl_a[1])[chl_test] #keeping only the non-nan values in Chl_a
#print(chl_test2)
#chl_test2=np.mask(np.array(fd.Chl_a[1]))
#print(chl_test2)

#### Masked array (1) - isfinite

In [8]:
chl_a_masked=np.ma.array(fd.Chl_a[0])
#print(chl_a_masked)
#np.ma.isMaskedArray(chl_a_masked)
chl_a_masked.giraffe=(np.isfinite(chl_a_masked)) 
#print(chl_a_masked.mask)
t1=chl_a_masked
#print(t1.giraffe)
#display(t1.giraffe)
msktest= t1.giraffe
display(np.ma.array(fd.Depth[0]))
depthmaskify=np.ma.array(fd.Depth[0])[msktest]
display(depthmaskify)

masked_array(data=[1576.696, 1478.468, 1380.657, 1282.416, 1184.159,
                   1085.855,  987.781,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,  939.274,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,  890.41 ,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      nan,      nan,      nan,      nan,
                        nan,      

masked_array(data=[1576.696, 1478.468, 1380.657, 1282.416, 1184.159,
                   1085.855,  987.781,  939.274,  890.41 ,  841.229,
                    792.155,  742.556,  693.024,  643.412,  594.755,
                    545.237,  496.033,  445.958,  397.175,  376.876,
                    357.109,  347.412,  337.735,  327.761,  315.512,
                    306.347,  297.687,  287.888,  278.455,  268.013,
                    258.51 ,  248.394,  238.386,  229.267,  218.714,
                    209.278,  198.852,  188.901,  179.216,  169.62 ,
                    159.647,  147.24 ,  137.811,  129.924,  119.406,
                    110.42 ,  100.277,   92.438,   84.392,   76.741,
                     69.536,   62.726,   56.519,   51.303,   46.779,
                     41.661,   35.711,   30.593,   25.376,   19.99 ,
                     14.535,    8.475],
             mask=False,
       fill_value=1e+20)

#### Masked array (2) - isnan 

In [13]:
chl_a_masked=np.ma.array(fd.Chl_a[0])
chl_a_masked.giraffe=(~np.isnan(chl_a_masked)) 
t1=chl_a_masked
#print(t1.giraffe)
#display(t1.giraffe)
msktest= t1.giraffe
#display(np.ma.array(fd.Depth[0]))
depthmaskify=np.ma.array(fd.Depth[0])[msktest]
display(depthmaskify)

masked_array(data=[1576.696, 1478.468, 1380.657, 1282.416, 1184.159,
                   1085.855,  987.781,  939.274,  890.41 ,  841.229,
                    792.155,  742.556,  693.024,  643.412,  594.755,
                    545.237,  496.033,  445.958,  397.175,  376.876,
                    357.109,  347.412,  337.735,  327.761,  315.512,
                    306.347,  297.687,  287.888,  278.455,  268.013,
                    258.51 ,  248.394,  238.386,  229.267,  218.714,
                    209.278,  198.852,  188.901,  179.216,  169.62 ,
                    159.647,  147.24 ,  137.811,  129.924,  119.406,
                    110.42 ,  100.277,   92.438,   84.392,   76.741,
                     69.536,   62.726,   56.519,   51.303,   46.779,
                     41.661,   35.711,   30.593,   25.376,   19.99 ,
                     14.535,    8.475],
             mask=False,
       fill_value=1e+20)

In [21]:
#depthmaskify=chl_a_masked[chl_a_masked.giraffe]
print(chl_a_masked)
depthmaskify=fd.Chl_a[0,:][chl_a_masked.giraffe]
display(fd.Chl_a[0,:])
display(depthmaskify)


[0.     0.0036 0.     0.0036 0.0036 0.0036 0.0036    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan 0.        nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan 0.        nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan 0.0036    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
 0.0036    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan 0.0036    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    na

<xarray.DataArray 'Chl_a' (N_LEVELS: 558)>
dask.array<getitem, shape=(558,), dtype=float64, chunksize=(558,), chunktype=numpy.ndarray>
Dimensions without coordinates: N_LEVELS
Attributes:
    units:    mg/m^3

<xarray.DataArray 'Chl_a' (N_LEVELS: 62)>
dask.array<getitem, shape=(62,), dtype=float64, chunksize=(62,), chunktype=numpy.ndarray>
Dimensions without coordinates: N_LEVELS
Attributes:
    units:    mg/m^3

## Testing

In [26]:
yvals_int = []
yvals=fd.Chl_a[:,::-1]
xvals=fd.Depth[:,::-1]
n=1
yvalmasked=np.ma.array(yvals[n,:])
yvalmasked.mask=np.isfinite(yvalmasked)
yvalsclean=yvals[n,:][yvalmasked.mask]
xvalsclean=xvals[n,:][yvalmasked.mask]
yvals_int.append(np.interp(x_int, xvalsclean, yvalsclean))
print(yvals_int)

## QC flags

In [41]:
fd.Depth_QF[0].values.astype(float)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.